## Transcribe content, video ##

In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

# Set the OpenAI API key
openai.api_key = ""

prompt = """
This is a recording of a conversation between 5 people and 2 companies.
Please transcribe the audio in the following format:

'Person 1: <utterance>
Person 2: <utterance>
...
"""

# Open the audio file
audio_file = open("Vmcall.m4a", "rb")

# Transcribe the audio
transcript = openai.Audio.transcribe("whisper-1", audio_file, prompt=prompt)

# Convert the transcript to a string
transcript_string = str(transcript)

# Print the transcript
# (transcript)
# Save the transcript to a file
with open("transcript.txt", "w") as f:
    f.write(transcript_string)

## Summarize the transcription ##

In [ ]:
import openai
import textwrap

# Set the OpenAI API key
openai.api_key = ""  # Replace with your actual key

# User-provided information
meeting_attendees = input("Please enter the meeting attendees: ")
company_name = input("Please enter the company name: ")
main_topic = input("Please enter the main topic: ")

# Open the transcript file
with open("../data/transcript.txt", "r") as f:
    transcript_string = f.read()

# Define the size of chunks
chunk_size = 2048  # Adjusted to be compatible with the new model's token limit

# Split the transcript into chunks
chunks = textwrap.wrap(transcript_string, chunk_size)

# Open the output file
with open("../data/summary.txt", "w") as out_file:
    # Write the user-provided information to the file
    out_file.write(f"Meeting Attendees: {meeting_attendees}\n")
    out_file.write(f"Company Name: {company_name}\n")
    out_file.write(f"Main Topic: {main_topic}\n\n")

    for idx, chunk in enumerate(chunks):
        # Feed the chunk to the model in a conversation
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system", 
                    "content": f"You are a helpful assistant that summarizes text, writes action items and emails. This text is from a meeting on the topic '{main_topic}' at '{company_name}', attended by {meeting_attendees}. Please summarize the following text: {chunk}"
                },
            ]
        )

        # Write the summary of each chunk to the file
        summary = response['choices'][0]['message']['content']
        out_file.write(f"Summary of chunk {idx+1}:\n{summary}\n\n")


## QNA with data ##

In [ ]:
import openai

# Set the OpenAI API key
openai.api_key = ""  # Replace with your actual key

# Open the summary file and read it
with open("../data/summary.txt", "r") as f:
    summary_string = f.read()

# Initialize conversation history with the summary
conversation_history = [
    {"role": "system", "content": "You are a helpful assistant. Be short and concise with responses."},
    {"role": "user", "content": f"This is the summary of the meeting: {summary_string}"},
]

# After writing all summaries, let the user ask questions
while True:
    user_question = input("You can ask a question about the summarized text. Type 'quit' to end: ")
    if user_question.lower() == 'quit':
        break

    # Append the user question to the conversation history
    conversation_history.append({"role": "user", "content": user_question})

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation_history
    )

    # Get the assistant's message from the response and print it
    answer = response['choices'][0]['message']['content']
    print("Answer: ", answer)

    # Append the assistant's answer to the conversation history
    conversation_history.append({"role": "assistant", "content": answer})
